
Nom: MENDY
Prenom: Elie
ID: 19004664





# Partie 1: texte

Soit le code de départ suivant, tiré du cours:

In [ ]:
stoplist = []

def pilote(fichier, idx):
    with open(fichier, 'r', encoding='utf8') as fp:
        # on itere sur le fichier ligne par ligne
        for i, ligne in enumerate(fp):
            indexe(idx, ligne.split(), i)

def ajoute(idx, mot, ligne):
    # si le mot ne se trouve pas encore dans l'index, 
    # initialise les occurences par une liste vide
    if mot not in idx:
        idx[mot] = []
    # si la ligne n'a pas encore ete enregistree pour ce mot
    if ligne not in idx[mot]:
        # on ajoute le numero de ligne correspondant au mot
        idx[mot].append(ligne + 1)

def indexe(idx, mots, ligne):
    # on itere sur les mots
    for mot in mots:
        # met obligatoirement en minuscule
        mot = mot.lower()
        # nettoyage du mot, voir ci-apres
        mot = nettoie(mot)
        # on en profite pour verifier si on veut 
        # vraiment indexer ce mot
        if mot and mot not in stoplist:
            ajoute(idx, mot, ligne)

def nettoie(mot):
    return mot

def prd(idx):
    for mot in sorted(idx):
        print('\t', mot, ':', idx[mot])

index = {}
pilote('../cigale_fourmi.txt', index)
prd(index)

## Stoplist

Compléter la `stoplist` de façon à ne garder que les mots potentiellement intéressants. Précisez votre méthodologie.

In [1]:
def nettoie(mot):
    if "aujourd'hui" in mot: return "aujourdh'hui"
    if "'" in mot:  mot = mot.split("'")[1]     # nettoyage de l'apostrophe avant un mot
    caracteres = []                             # liste 'caracteres' servant a construire le mot nettoyé
    for lettre in mot:                          # pour chaques symboles du mot
        if lettre not in '(,.!?;\'")':               # si ce n'est pas un symbole de ponctuation
            caracteres.append(lettre)               # on ajoute ce caractere a la list 'caracteres'
    return "".join(caracteres)                  # on retourne le mot nettoyé


In [2]:
# Méthodologie expliquée dans le document readme.pdf joint

stoplist = "le la les un une des du de ce cet cette ces mon ton son ma ta sa notre votre leur mes tes ses nos vos leurs quel quelle quelles quels je tu il nous vous ils elle elles me te se qui que quoi à ne mais ou est donc or ni car c'est eh".split()

## Nettoyage

Compléter le code de la fonction `nettoie` afin de ne pas tenir compte de la ponctuation au sein des mots.

Attention, il peut y avoir plusieurs signes de ponctuation avant ou après le mot.

In [3]:
## Ici les tests pour `nettoie`

for mot in [
    "aujourd'hui",
    "!dansez",
    "Cigale,",
    "l'Été,",
    "voisine,",
    "d'animal,",
    "l'Oût,",
    "?",# renverra "" une chaine de caractere vide qui ne sera pas ajoutée à l'indexe.
    "chantais,",
    "n'est"
]: print(nettoie(mot))



aujourdh'hui
dansez
Cigale
Été
voisine
animal
Oût

chantais
est


## Affichage

La fonction `prd` est assez basique en l’état. Modifier cette fonction pour que la liste de lignes soit présentée sous la forme: `programme : 1-2, 7` quand il y a 2 numéros consécutifs ou plus; par exemple, 1, 2, 3, 5, la liste est présentée sous la
forme `1-3, 5`.

Pour ce faire, coder une fonction `presente` prenant une liste d'entier et retournant une chaîne de caractères au format voulu.

In [4]:

def presente(lst):
    debut = lst[0]          # debut d'un interval (par defaut égal a la première ligne ou se trouve le mot)
    fin = lst[0]            # fin d'un interval
    intervalle =[""]        # creation de la liste des intervalles a renvoyer

    for ligne in lst:                                          # pour chaques lignes ecrites dans la liste
        if ligne == debut:                                             # si la ligne est le début de l'interval traité
            intervalle[-1] = str(debut)                                   # on initialise un nouvel interval
        elif ligne == fin + 1:                                         # sinon si on est a la ligne qui suivante
            fin = ligne                                                   # la var 'fin' prend la valeur de 'ligne'
            intervalle[-1] = '{}-{}'.format(debut, fin)                   # on agrandi l'interval que l'on construit,
        else:                                                          # sinon
            fin  = ligne                                                  # la var 'fin' prend la valeur de 'ligne'
            debut = ligne                                                 # la var 'debut' prend la valeur de 'ligne'
            if debut == fin:                                              # si l'intervalle ne fait qu'une ligne
                intervalle.append(str(debut))                                # on l'ajoute a la liste 'intervalle' tel une chaine de caractere
            else:                                                         # si l'interval est plus consequent
                intervalle.append('{}-{}'.format(debut, fin))                # on ajoute l'intervale créé dans la liste 'intervalle' dans le format souhaité "debut-fin"

    return ", ".join(intervalle)    # on retourne la liste de nos intervales de lignes au format souhaité


In [5]:
## Ici, les tests pour `presente`

for i in [
    [12, 18],
    [1, 2, 3, 5],
    [1, 2, 3, 4 ,5, 6, 8, 10, 11, 12, 78],
    [96 ,97 ,98, 99, 100, 101, 256, 257, 258, 259, 260],
    [5, 6, 7, 8, 10, 11, 12, 13, 16, 17, 18, 19, 20]
]: print(presente(i))

12, 18
1-3, 5
1-6, 8, 10-12, 78
96-101, 256-260
5-8, 10-13, 16-20


# Partie 2: web

L'objet de cette partie est de modifier et d'adapter le code obtenu à la partie 1 pour le transformer en robot capable d'indexer des pages web.


## Extraction des liens

A l'aide des indications vues dans le cours, écrire une fonction `extraire_liens(page)` retournant la liste des liens HTTP présents dans le contenu de la page. Pour rappel, la structure d'une balise contenant un lien est:

```
<a href="http..." ...>
```

In [6]:
def extraire_liens(page):
    """ fonction utilisée pour l'extractions
        des liens présent sur une page web """

    for ligne in page.split('\n'):
        x = ligne.find('href=')                                                 # utilisation de find() pour chercher un lien
        if x < 0 : pass
        ref = ligne[x+6:].split('"')[0]                                         # capture du lien dans la variable ref
        if ref.startswith('https:') and ref not in biblioteque_liens:           # ajout du lien dans la bibliotèque de liens
            biblioteque_liens.append(ref)



#### les tests pour `extraire_liens`

dans le script:
supprimer les '#' de l'option: 

"recuperer liste des liens collectés dans un fichier"


## Nettoyage de la page

On souhaite nettoyer la page des différentes balises HTML qui s'y trouvent (une balise commence par le caractère `'<'` et termine par `'>'`). Vous devez réaliser une fonction `nettoie_page(page)` dont le rôle est de supprimer les balises HTML du contenu.

In [ ]:
def nettoie_page(page):
    """ fonction utilisée pour le nettoyage
        du contenu d'une page web """

    tag = False
    page_nettoyee = ""
    for c in page:
        if c == '<': tag = True                             # caractere = '<'     tag on
        elif c == '>':                                      # caractere = '>'
            tag = False                                         # tag off
            page_nettoyee += " "                                # on ajoute un espace au début du contenu à ajouter
                                                                # pour eviter la concatenation de deux mots
        if not tag: page_nettoyee += c                      # not tag
    return page_nettoyee


## Adaptation du code

Adapter la fonction `pilote` vue ci-dessus pour prendre en charge des pages web au lieu de fichiers sur le disque. Pour cela, vous utiliserez `requests` afin d'ouvrir les URL passées en argument à la fonction.

Ne pas perdre de vue que c'est désormais l'URL qui doit être indexée et non le numéro de ligne. En revanche, les fonctions `indexe` et `ajoute` ne devraient pas subir de modification trop conséquente.

Enfin, intégrer le nettoyage des pages avant l'indexage.


## Finalisation du robot

Une fois que le code fonctionne, mettre en place l'extraction des liens d'une page et leur visite par le robot. Ainsi, à partir d'une page donnée, le robot doit être capable de suivre les hyperliens rencontrés et continuer l'indexation des nouvelles pages détectées.

Vous prendrez soin en particulier:
1. attention à bien éviter les cycles dans la visite des pages.
2. limiter le nombre de pages web explorées par le robot.

A présent, vous devriez avoir un robot virtuellement capable d'indexer le web!  
N'oubliez pas de rendre votre script accompagné de quelques exemples.

__Mettez le tout dans un script python dédié (sans oublier votre nom, prénom et numéro d’étudiant).__